In [8]:
!pip install faiss-cpu sentence-transformers yandexcloud grpcio yandex-cloud-ml-sdk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 3.3 MB/s eta 0:00:00


In [54]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from yandex_cloud_ml_sdk import YCloudML
from typing import Dict, List, Optional, Any
import uuid
import datetime
from dataclasses import dataclass
from enum import Enum
import json
import os

class YandexGPTClient:
    def __init__(self, api_key: str, folder_id: str):
        self.sdk = YCloudML(
            folder_id=folder_id,
            auth=api_key
        )
        self.model = self.sdk.models.completions("yandexgpt", model_version="rc")
        self.model = self.model.configure(temperature=0.6, max_tokens=2000)

    def generate_response(self, prompt: str) -> str:
        messages = [
            {"role": "system", "text": "Ты - помощник по умному термостату. Отвечай на вопросы кратко и по делу."},
            {"role": "user", "text": prompt}
        ]

        try:
            result = self.model.run(messages)
            if hasattr(result[0], 'text'):
                return result[0].text
            elif hasattr(result[0], 'message') and hasattr(result[0].message, 'text'):
                return result[0].message.text
            elif hasattr(result[0], 'alternatives') and len(result[0].alternatives) > 0:
                return result[0].alternatives[0].message.text
            else:
                return "Не удалось обработать ответ от модели."
        except Exception as e:
            return f"Ошибка при генерации ответа: {str(e)}"

class FAISSVectorStore:
    def __init__(self, dimension: int = 384):
        self.dimension = dimension
        self.index = faiss.IndexFlatL2(dimension)
        self.documents = []
        self.encoder = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

    def add_documents(self, documents: List[Dict[str, Any]]):
        texts = [doc['text'] for doc in documents]
        embeddings = self.encoder.encode(texts, show_progress_bar=True)
        self.index.add(np.array(embeddings).astype('float32'))
        self.documents.extend(documents)

    def search(self, query: str, k: int = 3) -> List[Dict[str, Any]]:
        query_embedding = self.encoder.encode([query])
        distances, indices = self.index.search(np.array(query_embedding).astype('float32'), k)
        return [self.documents[i] for i in indices[0]]

class DialogManager:
    def __init__(self, vector_store: FAISSVectorStore, llm_client: YandexGPTClient):
        self.vector_store = vector_store
        self.llm_client = llm_client
        self.active_dialogs = {}
        self.dialog_templates = self._load_dialog_templates()
        self.tickets_dir = "support_tickets"
        if not os.path.exists(self.tickets_dir):
            os.makedirs(self.tickets_dir)

    def _load_dialog_templates(self) -> Dict[str, Any]:
        return {
            "thermostat_diagnosis": {
                "steps": [
                    {
                        "id": "welcome",
                        "prompt": "Привет! Я помогу тебе с настройкой термостата. Какую проблему ты наблюдаешь?",
                        "variable": None,
                        "next_step": "current_temp"
                    },
                    {
                        "id": "current_temp",
                        "prompt": "Какая температура сейчас в комнате?",
                        "variable": "current_temp",
                        "next_step": "desired_temp"
                    },
                    {
                        "id": "desired_temp",
                        "prompt": "Какая температура должна быть в комнате?",
                        "variable": "desired_temp",
                        "next_step": "time_of_day"
                    },
                    {
                        "id": "time_of_day",
                        "prompt": "Когда это произошло? Утром, днем или вечером?",
                        "variable": "time_of_day",
                        "next_step": {
                            "condition": "check_duration",
                            "function": self._check_duration_condition
                        }
                    },
                    {
                        "id": "create_ticket",
                        "prompt": "Хотите создать заявку на техподдержку?",
                        "variable": "create_ticket",
                        "next_step": {
                            "condition": "user_choice",
                            "options": {
                                "Да": "ticket_created",
                                "Нет": "end_conversation"
                            }
                        }
                    },
                    {
                        "id": "wait_hour",
                        "prompt": "Подождите в течение часа, если проблема продолжает наблюдаться - снова обратитесь ко мне.",
                        "next_step": "end_conversation"
                    },
                    {
                        "id": "ticket_created",
                        "prompt": "Заявка создана. Техподдержка свяжется с вами в ближайшее время.",
                        "next_step": "end_conversation"
                    },
                    {
                        "id": "end_conversation",
                        "prompt": "Все готово! Если потребуется дополнительная помощь, обращайтесь.",
                        "next_step": None
                    }
                ]
            }
        }

    def _check_duration_condition(self, dialog_state: Dict[str, Any]) -> str:
        return "create_ticket"

    def start_new_dialog(self, dialog_type: str) -> str:
        dialog_id = str(uuid.uuid4())
        self.active_dialogs[dialog_id] = {
            "type": dialog_type,
            "current_step": "welcome",
            "variables": {},
            "history": [],
            "start_time": datetime.datetime.now(),
            "template": self.dialog_templates.get(dialog_type)
        }
        return dialog_id

    def _save_support_ticket(self, ticket_info: Dict[str, Any]) -> str:
        """Сохраняет информацию о заявке в файл"""
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        ticket_id = f"ticket_{timestamp}"

        dialog_history = []
        for entry in ticket_info.get("dialog_history", []):
            dialog_history.append({
                "step": entry["step"],
                "user_input": entry["user_input"],
                "timestamp": entry["timestamp"].isoformat() if isinstance(entry["timestamp"], datetime.datetime) else entry["timestamp"]
            })

        ticket_data = {
            "ticket_id": ticket_id,
            "status": "new",
            "created_at": datetime.datetime.now().isoformat(),
            "problem_details": {
                "current_temp": ticket_info.get("current_temp"),
                "desired_temp": ticket_info.get("desired_temp"),
                "time_of_day": ticket_info.get("time_of_day"),
                "duration": "более часа",
            },
            "dialog_history": dialog_history,
            "device_info": {
                "type": "smart_thermostat",
                "error_state": ticket_info.get("error_state", False)
            }
        }

        # сохраняем в JSON файл
        filename = os.path.join(self.tickets_dir, f"{ticket_id}.json")
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(ticket_data, f, ensure_ascii=False, indent=2)

        return ticket_id

    def process_user_input(self, dialog_id: str, user_input: str) -> str:
        if dialog_id not in self.active_dialogs:
            return "Диалог не найден. Пожалуйста, начните новый диалог."

        # проверка на некорректный ввод
        if not user_input or any(word in user_input.lower() for word in ["хуй", "fuck", "блять"]):
            return "Пожалуйста, давайте общаться вежливо. Я здесь, чтобы помочь вам с термостатом."

        dialog_state = self.active_dialogs[dialog_id]
        current_step_id = dialog_state["current_step"]
        variables = dialog_state["variables"]

        # сохраняем историю
        dialog_state["history"].append({
            "step": current_step_id,
            "user_input": user_input,
            "timestamp": datetime.datetime.now()
        })

        def extract_temperature(text: str) -> Optional[float]:
            try:
                if "error" in text.lower() or "нет температуры" in text.lower():
                    return None

                text = text.lower()
                text = text.replace("градуса", "").replace("°c", "")
                text = text.replace("температура", "").replace("-", "")
                text = text.replace("текущая", "").replace("желаемая", "")
                text = text.strip()
                temp = float(text)

                # проверка на реалистичность температуры
                if temp > 100:  # явно нереальная температура
                    return None
                return temp
            except ValueError:
                return None

        # Обработка шагов диалога
        if current_step_id == "welcome":
            dialog_state["current_step"] = "current_temp"
            return ("Какая температура сейчас в комнате? \n"
                    "Если термостат показывает ошибку, пожалуйста, введите примерную температуру в помещении.")

        elif current_step_id == "current_temp":
            temp = extract_temperature(user_input)
            if temp is not None and 10 <= temp <= 35:
                variables["current_temp"] = temp
                dialog_state["current_step"] = "desired_temp"
                return "Какая температура должна быть в комнате?"
            else:
                return ("Пожалуйста, введите температуру числом от 10 до 35 градусов.\n"
                       "Например: 22 или 23.5")

        elif current_step_id == "desired_temp":
            temp = extract_temperature(user_input)
            if temp is not None and 10 <= temp <= 35:
                variables["desired_temp"] = temp
                dialog_state["current_step"] = "time_of_day"
                return "Когда это произошло? Утром, днем или вечером?"
            else:
                return ("Пожалуйста, введите желаемую температуру числом от 10 до 35 градусов.\n"
                       "Например: 22 или 23.5")

        elif current_step_id == "time_of_day":
            valid_times = {"утром": "утром", "днем": "днем", "вечером": "вечером",
                          "утро": "утром", "день": "днем", "вечер": "вечером",
                          "сейчас": "сейчас"}
            user_time = user_input.lower()

            # Определяем время суток для "сейчас"
            if user_time == "сейчас":
                hour = datetime.datetime.now().hour
                if 5 <= hour < 12:
                    user_time = "утром"
                elif 12 <= hour < 18:
                    user_time = "днем"
                else:
                    user_time = "вечером"

            for time_key, time_value in valid_times.items():
                if time_key in user_time:
                    variables["time_of_day"] = time_value
                    dialog_state["current_step"] = "duration_check"
                    return "Проблема наблюдается более часа?"
            return ("Пожалуйста, укажите время суток: утром, днем или вечером\n"
                   "Например: утром или днем")

        elif current_step_id == "duration_check":
            positive_answers = ["да", "yes", "более", "больше"]
            negative_answers = ["нет", "no", "менее", "меньше"]

            user_answer = user_input.lower()
            if any(ans in user_answer for ans in positive_answers):
                dialog_state["current_step"] = "create_ticket"
                return "Хотите создать заявку на техподдержку?"
            elif any(ans in user_answer for ans in negative_answers):
                dialog_state["current_step"] = "wait_hour"
                return ("Подождите в течение часа, если проблема продолжает наблюдаться - "
                       "снова обратитесь ко мне, и мы создадим заявку в техподдержку.")
            else:
                return "Пожалуйста, ответьте да или нет"

        elif current_step_id == "wait_hour":
            # Если пользователь пишет, что проблема осталась
            if "проблема" in user_input.lower() and ("осталась" in user_input.lower() or "продолжается" in user_input.lower()):
                dialog_state["current_step"] = "create_ticket"
                return "Хотите создать заявку на техподдержку?"
            dialog_state["current_step"] = "end_conversation"
            return "Все готово! Если проблема останется, обратитесь снова, и мы создадим заявку в техподдержку."

        elif current_step_id == "create_ticket":
            positive_answers = ["да", "yes", "конечно", "хочу"]
            if any(ans in user_input.lower() for ans in positive_answers):
                # оформление тикета
                ticket_info = {
                    "current_temp": variables.get("current_temp"),
                    "desired_temp": variables.get("desired_temp"),
                    "time_of_day": variables.get("time_of_day"),
                    "timestamp": datetime.datetime.now().isoformat(),
                    "dialog_history": dialog_state["history"],
                    "error_state": any("error" in str(h.get("user_input")).lower()
                                     for h in dialog_state["history"])
                }

                # сохранение тикета
                ticket_id = self._save_support_ticket(ticket_info)
                variables["ticket_info"] = ticket_info
                variables["ticket_id"] = ticket_id

                dialog_state["current_step"] = "ticket_created"
                return (f"Заявка #{ticket_id} создана.\n"
                       f"Текущая температура: {variables.get('current_temp')}°C\n"
                       f"Желаемая температура: {variables.get('desired_temp')}°C\n"
                       f"Время возникновения: {variables.get('time_of_day')}\n"
                       "Техподдержка свяжется с вами в ближайшее время.")
            else:
                dialog_state["current_step"] = "end_conversation"
                return "Хорошо. Если потребуется помощь - обращайтесь."

        elif current_step_id in ["ticket_created", "end_conversation"]:
            # затычка для непонятливого пользователя
            if "проблема" in user_input.lower():
                return "Чтобы начать новую диагностику, напишите 'начать'"
            return "Диалог завершен. Чтобы начать новую диагностику, напишите 'начать'"

        return "Извините, произошла ошибка. Чтобы начать заново, напишите 'начать'"

vector_store = FAISSVectorStore()
llm_client = YandexGPTClient(api_key="апишечка", folder_id="папочка")
dialog_manager = DialogManager(vector_store, llm_client)

documents = [
    {"text": "Умный термостат не поддерживает температуру. Возможные причины: неисправность датчика, проблемы с питанием."},
    {"text": "Для создания заявки на техподдержку термостата необходимо указать текущую и желаемую температуру."},
    {"text": "Среднее время реакции техподдержки по заявкам - 1 час в рабочее время."}
]
vector_store.add_documents(documents)

dialog_id = dialog_manager.start_new_dialog("thermostat_diagnosis")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [55]:
while True:
  s = input(str())
  print(s)
  print(dialog_manager.process_user_input(dialog_id, s))


привет
привет
Какая температура сейчас в комнате? 
Если термостат показывает ошибку, пожалуйста, введите примерную температуру в помещении.
23
23
Какая температура должна быть в комнате?
24
24
Когда это произошло? Утром, днем или вечером?
утром
утром
Проблема наблюдается более часа?
да
да
Хотите создать заявку на техподдержку?
да
да
Заявка #ticket_20250403_194916 создана.
Текущая температура: 23.0°C
Желаемая температура: 24.0°C
Время возникновения: утром
Техподдержка свяжется с вами в ближайшее время.
спасибо
спасибо
Диалог завершен. Чтобы начать новую диагностику, напишите 'начать'


KeyboardInterrupt: Interrupted by user